In [164]:
import pandas as pd
import numpy as np
import json
import re
import matplotlib.pyplot as plt
import spacy

from sklearn.model_selection import train_test_split

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Embedding, Input, Bidirectional, LSTM, concatenate

In [2]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 45.4 MB 12.7 MB/s eta 0:00:01   |███▍                            | 4.8 MB 2.8 MB/s eta 0:00:15��████████        | 34.1 MB 65.7 MB/s eta 0:00:01██████████████▋       | 34.9 MB 65.7 MB/s eta 0:00:01███████████████       | 35.4 MB 65.7 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
# Load first file
df = pd.read_csv('Kickstarter.csv')

In [4]:
# Loop through 8 files and concat to original DF
for i in range(8):
  df_ = pd.read_csv(f'Kickstarter00{i+1}.csv')
  df = pd.concat([df, df_])

In [5]:
def clean_data(df):

  # Removing duplicate entries then set 'id' as index
  df.drop_duplicates(subset='id', inplace=True)
  df.set_index('id', inplace=True)

  # Drop columns with 99% null values
  df.drop(columns=['friends', 'is_backing', 'is_starred', 'permissions'], inplace=True)

  # Drop rows where state is not 'successful' or 'failed'.  We are looking at binary outcomes
  df = df[(df['state'] == 'successful')|(df['state'] == 'failed')]

  # Dropping high cardinality, redundant, and uninteresting columns
  df = df.drop(columns=['country_displayable_name', 'creator', 'currency_symbol', 'name', 'photo', 'profile', 'source_url', 'urls', 'usd_type'])

  # Dropping columns with only 1 unique value
  df = df.drop(columns=['disable_communication', 'is_starrable'])

  # Dropping leaky columns and currency exchange columns
  df = df.drop(columns=['converted_pledged_amount', 'currency', 'currency_trailing_code', 'current_currency', 'fx_rate', 'pledged', 'static_usd_rate', 'usd_exchange_rate', 'usd_pledged'])

  # Creating 'campaign_length' feature
  df['campaign_length'] = df['deadline'] - df['launched_at']

  # Dropping columns which can't be tinkered by user
  df.drop(columns=['country', 'created_at', 'deadline', 'launched_at', 'state_changed_at', 'spotlight', 'location', 'slug', 'backers_count'], inplace=True)

  # Pull the category names out and store in a list
  dict_list = []
  for entry in df['category']:
    category = json.loads(entry)
    dict_list.append(category['name'])

  # Create new category column with just the category and not dictionaries
  df['cat'] = dict_list

  # Drop old category
  df.drop(columns='category', inplace=True)

  # Create 'word_count' feature
  description_lengths = [len(description.split()) for description in df['blurb']]
  df['word_count'] = description_lengths

  # Make 'staff_pick' column integers
  df['staff_pick'] = df['staff_pick'].astype('int64')

  # Re-order columns
  df = df[['blurb', 'cat', 'word_count', 'campaign_length', 'goal', 'staff_pick', 'state']]
  
  return df

In [6]:
df = clean_data(df)

In [7]:
df.head(2)

,blurb,cat,word_count,campaign_length,goal,staff_pick,state
id,,,,,,,
1837982762,Create 200 frames as animated asymmetric tesse...,Conceptual Art,17,2592000,2000.0,0,failed
1820905478,SixNip will be recording its debut album start...,Rock,15,3024000,420.0,0,successful


In [8]:
cat_dict = {}
for i, cat in enumerate(df['cat'].unique()):
    cat_dict[cat] = i

In [9]:
df['cat'] = df['cat'].map(cat_dict)

In [10]:
df.head(2)

,blurb,cat,word_count,campaign_length,goal,staff_pick,state
id,,,,,,,
1837982762,Create 200 frames as animated asymmetric tesse...,0,17,2592000,2000.0,0,failed
1820905478,SixNip will be recording its debut album start...,1,15,3024000,420.0,0,successful


In [11]:
# Pull out target variable
y = df['state']

In [12]:
# Convert target variable to numeric labels
y = y.map({'successful': 1, 'failed': 0})

In [13]:
# Creating Feature Matrix by dropping target variable
X = df.drop(columns='state')

In [14]:
def clean_text(text):
    """
    Accepts a single text document and performs several regex substitutions in order to clean the document. 
    
    Parameters
    ----------
    text: string or object 
    
    Returns
    -------
    text: string or object
    """
    
    # order of operations - apply the expression from top to bottom
    non_alpha = '[^a-zA-Z]'
    multi_white_spaces = "[ ]{2,}"
    single_letter_words = '(\s[a-zA-Z]\s)'
    
    text = re.sub(non_alpha, ' ', text)
    text = re.sub(single_letter_words, ' ', text)
    text = re.sub(single_letter_words, ' ', text)
    text = re.sub(multi_white_spaces, " ", text)
    
    
    # apply case normalization 
    return text.lower().lstrip().rstrip()

def tokenize(document):
    """
    Takes a doc and returns a string of lemmas after removing stop words.
    """
    
    doc = nlp(document)
    
    tokens = [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True) and (len(token) > 2)]
    return ' '.join(tokens)

In [15]:
X_clean = [clean_text(text) for text in X['blurb']]

In [16]:
nlp = spacy.load("en_core_web_md")

In [17]:
X_token = [tokenize(text) for text in X_clean]

In [127]:
def get_word_vectors(doc):
    """
    This serves as both our tokenizer and vectorizer. 
    Returns a list of word vectors, i.e. our doc-term matrix
    """
    return nlp(doc).vector

In [128]:
X_vect = []
for i, text in enumerate(X_token):
  X_vect.append(get_word_vectors(text))
  if i % 100 == 0:
    print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [133]:
X_vect = np.array(X_vect)

In [142]:
X_vect.shape

(28921, 300)

In [151]:
X_vect_test = []
for arr in X_vect:
    X_vect_test.append(np.expand_dims(arr, axis=1))

In [153]:
X_vect_test = np.array(X_vect_test)

In [154]:
X_vect_test.shape

(28921, 300, 1)

In [137]:
X_meta = df.drop(columns=['blurb', 'state'])

In [138]:
X_meta.shape

(28921, 5)

In [171]:
X_meta = np.array(X_meta)

In [174]:
X_meta_test = []
for arr in X_meta:
    X_meta_test.append(np.expand_dims(arr, axis=1))

In [176]:
X_meta_test = np.array(X_meta_test)

In [177]:
X_meta_test.shape

(28921, 5, 1)

In [165]:
nlp_input = Input(shape=(300, 1))
meta_input = Input(shape=(5, 1))

forward_layer = LSTM(128, return_sequences=True)
backward_layer = LSTM(128, activation='relu', return_sequences=True, go_backwards=True)

meta_out = Bidirectional(forward_layer, backward_layer=backward_layer)(meta_input)
nlp_out = Bidirectional(forward_layer, backward_layer=backward_layer)(nlp_input)

concat = concatenate([nlp_out, meta_out], axis=1)
classifier = Dense(32, activation='relu')(concat)
output = Dense(1, activation='sigmoid')(classifier)

model = Model(inputs=[nlp_input , meta_input], outputs=[output])

In [166]:
model.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['accuracy'] 
)

In [178]:
model.fit(
    x=[X_vect_test, X_meta_test],
    y=y,
    validation_split=0.2,
    shuffle=True,
    batch_size=32,
    epochs=10,
    class_weight={0: 0.34, 1: 0.66},
    workers=-1,
    
)

2021-09-28 16:43:53.935546: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
723/723 [==============================] - 392s 532ms/step - loss: 0.7192 - accuracy: 0.6556 - val_loss: 0.7116 - val_accuracy: 0.6871
Epoch 2/10
723/723 [==============================] - 342s 472ms/step - loss: 0.7336 - accuracy: 0.6549 - val_loss: 0.7338 - val_accuracy: 0.6845
Epoch 3/10
723/723 [==============================] - 570s 789ms/step - loss: 0.7120 - accuracy: 0.6563 - val_loss: 0.6744 - val_accuracy: 0.6895
Epoch 4/10
723/723 [==============================] - 438s 606ms/step - loss: 0.7068 - accuracy: 0.6566 - val_loss: 0.7529 - val_accuracy: 0.6846
Epoch 5/10
723/723 [==============================] - 420s 581ms/step - loss: 0.7055 - accuracy: 0.6567 - val_loss: 0.6719 - val_accuracy: 0.6895
Epoch 6/10
723/723 [==============================] - 401s 554ms/step - loss: 0.7244 - accuracy: 0.6550 - val_loss: 0.6980 - val_accuracy: 0.6871
Epoch 7/10
723/723 [==============================] - 431s 597ms/step - loss: 0.7274 - accuracy: 0.6545 - val_loss: 0.6996 -